## 🧼 Notebook 1: Preprocesamiento de reseñas

En esta notebook se realiza la carga, limpieza y normalización inicial de los datos de reseñas recolectadas mediante scraping. Se unifican los datasets, se corrigen las fechas, se limpian los textos y se eliminan duplicados o valores nulos antes del análisis exploratorio.

## 1. Imports y configuración general

En esta sección se importan las librerías necesarias para el procesamiento de datos de las reseñas.  
Incluye herramientas para:
- manipular archivos y estructuras (`os`, `json`, `pandas`),
- limpiar y normalizar texto (`re`, `unidecode`),
- y tratar fechas con formatos mixtos (`datetime`, `dateparser`, `locale`).


In [22]:
#1: Imports y configuración general

import pandas as pd
import json
from datetime import datetime
import re
from unidecode import unidecode
import os
from dateparser import parse
import locale 



## 2. Configuración del entorno y rutas

Se establecen las configuraciones iniciales del entorno:
- Se define el idioma local en español para interpretar fechas como "26 ago. 2024".
- Se declaran las rutas a los datos crudos y procesados. Estas rutas permiten guardar versiones intermedias o finales del dataset.


In [23]:
#Configuración de paths
data_raw_dir = '../data/raw'
data_processed_dir = '../data/processed'

#Configuración de idioma local
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')  

'es_ES.UTF-8'

## 3. Función de carga y limpieza de reseñas

La función `load_and_clean()` permite cargar reseñas desde un archivo `.json` y aplicar una limpieza básica.  
Incluye los siguientes pasos:

- **Carga del JSON** y conversión a DataFrame.
- **Normalización del texto** (pasaje a minúsculas, remoción de tildes y puntuación).
- **Conversión de fechas** desde formatos ISO o con nombres de meses en español, utilizando `dateparser`.
- Asignación del nombre del producto y retorno del DataFrame limpio.

Esta función permite reutilizar el mismo proceso para distintos productos.

In [24]:
def load_and_clean(json_file, product_name):
    """
    Carga datos JSON de reseñas, limpia texto, normaliza fechas y valida campos clave.
    """
    with open(os.path.join(data_raw_dir, json_file), 'r', encoding='utf-8') as f:
        raw_data = json.load(f)
    
    # Validar estructura mínima esperada
    data = [d for d in raw_data if all(k in d for k in ['text', 'rating', 'date'])]

    df = pd.DataFrame(data)
    df['producto'] = product_name

    # Limpieza de texto
    def clean_text(text):
        text = unidecode(text.lower())
        text = re.sub(r'[^\w\s]', '', text)
        return text
    df['text_clean'] = df['text'].apply(clean_text)

    # Conversión de fechas
    def parse_date(date_str):
        try:
            date_iso = pd.to_datetime(date_str, format='%Y-%m-%d', errors='coerce')
            if not pd.isna(date_iso):
                return date_iso
            return parse(date_str, languages=['es'], settings={'DATE_ORDER': 'DMY'})
        except:
            return pd.NaT
    df['date'] = df['date'].apply(parse_date)

    # Validaciones
    df = df[df['rating'].between(1, 5)]
    df.dropna(subset=['text', 'rating', 'date'], inplace=True)

    # Columnas auxiliares
    df['text_length'] = df['text_clean'].str.split().str.len()
    df['year_month'] = df['date'].dt.to_period('M')

    return df


### 3.1 Carga de reseñas de productos específicos

Aplicamos la función `load_and_clean()` a cada archivo `.json` de productos (Samsung A15 y Motorola G32).  
Esto nos permite mantener trazabilidad del origen de cada reseña y preparar los datos para su posterior unificación.


In [ ]:
df_samsung = load_and_clean('comentarios_Samsung_A15.json', 'Samsung A15')
df_motorola = load_and_clean('comentarios_Motorola_G32.json', 'Motorola G32')

## 4. Unificación, deduplicación y control de nulos

Una vez cargadas y limpiadas las reseñas de cada producto:

- Se concatenan en un único DataFrame (`df_total`).
- Se eliminan duplicados basados en el texto de la reseña.
- Se eliminan filas con valores nulos en campos clave como `text` y `rating`.

Se imprime un resumen de los valores nulos y se muestra un preview del DataFrame final.


In [ ]:
df_total = pd.concat([df_samsung, df_motorola], ignore_index=True)
df_total.drop_duplicates(subset=['text_clean', 'producto'], inplace=True)


### 4.1 Verificación del estado del DataFrame final

Se exploran aspectos clave del dataset:

- Cantidad de fechas faltantes.
- Cantidad total de valores nulos por columna.
- Distribución de las calificaciones (`rating`), útil para entender posibles sesgos.
- Estadísticas descriptivas generales (`describe()`), que permiten anticipar outliers o errores de carga.

Este chequeo asegura que los datos están listos para pasar al análisis exploratorio (EDA).


In [20]:
print("→ Nulos en fechas:", df_total['date'].isna().sum())
print("\n→ Nulos totales:\n", df_total.isna().sum())
print("\n→ Distribución de ratings:\n", df_total['rating'].value_counts().sort_index())
df_total.describe(include='all')


→ Nulos en fechas: 0

→ Nulos totales:
 text            0
rating          0
date            0
useful_votes    0
producto        0
text_clean      0
text_length     0
year_month      0
dtype: int64

→ Distribución de ratings:
 rating
1    116
2     65
3    187
4    366
5    351
Name: count, dtype: int64


,text,rating,date,useful_votes,producto,text_clean,text_length,year_month
count,1085,1085.000000,1085,1085.000000,1085,1085,1085.000000,1085
unique,1064,NaN,NaN,NaN,2,1063,NaN,29
top,,NaN,NaN,NaN,Samsung A15,,NaN,2025-04
freq,2,NaN,NaN,NaN,598,2,NaN,139
mean,NaN,3.710599,2024-08-20 14:49:12.995391488,4.490323,NaN,NaN,17.677419,NaN
min,NaN,1.000000,2022-12-28 00:00:00,0.000000,NaN,NaN,0.000000,NaN
25%,NaN,3.000000,2023-12-28 00:00:00,0.000000,NaN,NaN,5.000000,NaN
50%,NaN,4.000000,2024-11-11 00:00:00,0.000000,NaN,NaN,12.000000,NaN
75%,NaN,5.000000,2025-02-20 00:00:00,1.000000,NaN,NaN,22.000000,NaN
max,NaN,5.000000,2025-04-30 00:00:00,617.000000,NaN,NaN,198.000000,NaN


In [18]:
df_total.head()

,text,rating,date,useful_votes,producto,text_clean,text_length,year_month
0,El celular es precioso! se lo regale a mi mamá...,5,2024-09-19,433,Samsung A15,el celular es precioso se lo regale a mi mama ...,114,2024-09
1,Súper lindo el celu muy recomendado.,5,2024-07-11,94,Samsung A15,super lindo el celu muy recomendado,6,2024-07
2,"Esta muy lindo. La cámara linda ,la batería du...",5,2024-08-26,94,Samsung A15,esta muy lindo la camara linda la bateria dura...,32,2024-08
3,"Realmente el producto me encantó,publico algun...",5,2024-10-22,72,Samsung A15,realmente el producto me encantopublico alguna...,119,2024-10
4,Desde que conocí samsung no deje de usar desde...,5,2024-10-07,52,Samsung A15,desde que conoci samsung no deje de usar desde...,54,2024-10


## 5. Guardado del dataset limpio

Se exportan las reseñas unificadas y procesadas a un archivo `.csv` para análisis exploratorio posterior.  
Este archivo puede ser utilizado en notebooks siguientes para visualizaciones, NLP u otros análisis.

In [21]:
#Guardar datos procesados
os.makedirs(data_processed_dir, exist_ok=True)
output_path = os.path.join(data_processed_dir, 'reviews_unificado.csv')
df_total.to_csv(output_path, index=False, encoding='utf-8')
print(f"✅ Datos guardados en: {output_path}")
print(f"📊 Total de reseñas procesadas: {len(df_total)}")

✅ Datos guardados en: ../data/processed/reviews_unificado.csv
📊 Total de reseñas procesadas: 1085
